Импорт библиотек для анализа данных

In [50]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from catboost import CatBoostRegressor, Pool, cv

Загрузка датасета и просмотр первых строк

In [51]:
df_train = pd.read_csv("diamonds_train.csv")
print("Первые 5 строк обучающего датасета:")
print(df_train.head())

Первые 5 строк обучающего датасета:
   carat        cut color clarity  depth  table  price     x     y     z
0   0.51       Good     D     SI2   63.9   55.0   1180  5.04  5.10  3.24
1   0.72      Ideal     E     VS2   60.8   57.0   3091  5.79  5.82  3.53
2   0.70  Very Good     D    VVS2   62.8   60.0   4022  5.65  5.69  3.56
3   0.36      Ideal     D     SI1   61.2   57.0    663  4.59  4.63  2.82
4   0.54  Very Good     D     SI1   60.0   59.8   1593  5.30  5.34  3.18


Вывод информации о датасете

In [52]:
print("\nИнформация о датасете:")
df_train.info()


Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43018 entries, 0 to 43017
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    43018 non-null  float64
 1   cut      43018 non-null  object 
 2   color    43018 non-null  object 
 3   clarity  43018 non-null  object 
 4   depth    43018 non-null  float64
 5   table    43018 non-null  float64
 6   price    43018 non-null  int64  
 7   x        43018 non-null  float64
 8   y        43018 non-null  float64
 9   z        43018 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 3.3+ MB


Выделение целевой переменной

In [53]:
x = df_train.drop("price", axis=1)
y = df_train["price"]

Задание категориальных признаков

In [54]:
catboost_features = x.select_dtypes(include="object").columns.tolist()
print(f"\nКатегориальные признаки: {catboost_features}")


Категориальные признаки: ['cut', 'color', 'clarity']


Деление датасета на обучающий и валидационный

In [55]:
x_split, x_val, y_split, y_val = train_test_split(
    x, y, test_size=0.15, random_state=42, shuffle=True
)
print(f"\nРазмеры после разделения:")
print(f"x_split: {x_split.shape}")
print(f"x_val: {x_val.shape}")
print(f"y_split: {y_split.shape}")
print(f"y_val: {y_val.shape}")


Размеры после разделения:
x_split: (36565, 9)
x_val: (6453, 9)
y_split: (36565,)
y_val: (6453,)


Задание параметров обучения

In [56]:
params = {
    "iterations": 1000,
    "learning_rate": 0.05,
    "depth": 6,
    "l2_leaf_reg": 5,
    "loss_function": "RMSE",
    "eval_metric": "R2",
    "random_seed": 42,
    "verbose": 0,
    "thread_count": 4
}

Проведение кросс-валидации на 5-ти фолдах

In [57]:
print("\nКросс-валидация (5 фолдов):")

train_pool = Pool(x, y, cat_features=cat_features)

cv_results = cv(
    params=params,
    pool=train_pool,
    fold_count=5,
    shuffle=True,
    seed=42,
    plot=False
)


Кросс-валидация (5 фолдов):
Training on fold [0/5]

bestTest = 0.9822647465
bestIteration = 999

Training on fold [1/5]

bestTest = 0.9823393866
bestIteration = 970

Training on fold [2/5]

bestTest = 0.982245403
bestIteration = 999

Training on fold [3/5]

bestTest = 0.9823999549
bestIteration = 997

Training on fold [4/5]

bestTest = 0.9811557622
bestIteration = 999



In [58]:
print("Результаты кросс-валидации:")
print(f"Средний R²: {cv_results['test-R2-mean'].max():.4f}")

Результаты кросс-валидации:
Средний R²: 0.9821


Получаем значение лучшей итерации и коэффициента детерминации, выводим

In [59]:
best_iter_idx = cv_results["test-R2-mean"].idxmax()
best_r2 = cv_results.loc[best_iter_idx, "test-R2-mean"]
best_iteration = int(cv_results.loc[best_iter_idx, "iterations"])

print(f"\nЛучшая R²: {best_r2:.4f}")
print(f"Лучшая итерация: {best_iteration}")


Лучшая R²: 0.9821
Лучшая итерация: 999


Обучаем модель с учетом кросс-валидации

In [60]:
final_model = CatBoostRegressor(
    iterations=best_iteration,
    learning_rate=params["learning_rate"],
    depth=params["depth"],
    loss_function=params["loss_function"],
    eval_metric=params["eval_metric"],
    random_seed=params["random_seed"],
    verbose=200,
    cat_features=cat_features
)

получаем значения итоговой модели и выводим

In [63]:
final_model.fit(x_split, y_split)

y_val_pred = final_model.predict(x_val)

val_r2 = r2_score(y_val, y_val_pred)
val_rmse = mean_squared_error(y_val, y_val_pred)

print(f"\nРезультаты на валидационной выборке:")
print(f"R²: {val_r2:.4f}")
print(f"RMSE: {val_rmse:,.2f}")

0:	learn: 0.0832326	total: 26.4ms	remaining: 26.3s
200:	learn: 0.9791279	total: 5.11s	remaining: 20.3s
400:	learn: 0.9819831	total: 10.1s	remaining: 15.1s
600:	learn: 0.9836089	total: 15.2s	remaining: 10.1s
800:	learn: 0.9847433	total: 20.4s	remaining: 5.05s
998:	learn: 0.9856539	total: 25.9s	remaining: 0us

Результаты на валидационной выборке:
R²: 0.9820
RMSE: 290,459.80


Сохраняем модель

In [66]:
import joblib

joblib.dump(final_model, "final_model.pkl")

['final_model.pkl']

Загружаем тестовый датасет и модель при надобности

In [67]:
df_test = pd.read_csv("diamonds_test.csv")

# final_model = joblib.load("final_model.pkl")

Выводим первые значения датасета

In [68]:
print("diamonds_test.csv")
df_test.head()

diamonds_test.csv


,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,1.02,Good,F,SI2,59.2,58.0,6.51,6.56,3.87
1,1,0.70,Very Good,I,VVS1,59.5,58.0,5.78,5.81,3.45
2,2,0.32,Very Good,H,VVS2,63.4,56.0,4.37,4.34,2.76
3,3,0.42,Ideal,F,VVS2,62.2,56.0,4.79,4.82,2.99
4,4,0.40,Ideal,F,VS2,62.3,54.0,4.74,4.77,2.96


Получаем информацию о датасете

In [69]:
print("Информация о датасете: ")
df_test.info()

Информация о датасете: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5379 entries, 0 to 5378
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       5379 non-null   int64  
 1   carat    5379 non-null   float64
 2   cut      5379 non-null   object 
 3   color    5379 non-null   object 
 4   clarity  5379 non-null   object 
 5   depth    5379 non-null   float64
 6   table    5379 non-null   float64
 7   x        5379 non-null   float64
 8   y        5379 non-null   float64
 9   z        5379 non-null   float64
dtypes: float64(6), int64(1), object(3)
memory usage: 420.4+ KB


подготавливаем колонки, которые использовались в обучении

In [70]:
x_test = df_test[x.columns]

Получем предсказания значений цены в датасете

In [72]:
print("Предсказание на тестовом датасете...")
y_test_pred = final_model.predict(x_test)

Предсказание на тестовом датасете...


Сохраняем предсказания

In [73]:
submission = pd.DataFrame({
    "id": df_test["id"],
    "price": y_test_pred
})

submission.to_csv("submission.csv", index=False)
print("\nФайл 'submission.csv' успешно сохранен!")


Файл 'submission.csv' успешно сохранен!


Смотрим статистику датасета

In [74]:
print(f"\nСтатистика предсказаний:")
print(f"Количество предсказаний: {len(y_test_pred)}")
print(f"Минимальная цена: {y_test_pred.min():.2f}")
print(f"Максимальная цена: {y_test_pred.max():.2f}")
print(f"Средняя цена: {y_test_pred.mean():.2f}")
print(f"Медианная цена: {pd.Series(y_test_pred).median():.2f}")


Статистика предсказаний:
Количество предсказаний: 5379
Минимальная цена: 256.38
Максимальная цена: 18361.66
Средняя цена: 3934.49
Медианная цена: 2434.86
